# Semianr 4.1 - Applied Quantitative Logistics

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

## Binary Knapsak Problem

The Knapsack problem can be formulated as follows:

Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is maximized.

Let:
- `n` be the number of items
- `weight[i]` be the weight of item `i`
- `value[i]` be the value of item `i`
- `W` be the maximum weight capacity of the knapsack

The goal is to maximize the sum of the values of the items in the knapsack, subject to the constraint that the sum of the weights of the items cannot exceed `W`.



**Maximize:**

$$
z = \sum_{i=1}^{n} \text{value}[i] \times x[i]
$$

Subject to the constraint:

$$ 
\sum_{i=1}^{n} \text{weight}[i] \times x[i] \leq W 
$$

and

$$
x_i \in \{0, 1\}
$$

In [ ]:
# Let's consider 50 objects

# Values
np.random.seed(10)
v = list(np.random.randint(10, 90, size=50))

# Weights
np.random.seed(12)
w = list(np.random.randint(100, 900, size=50))

n = len(v)    # Number of Objects
W = 10000     # Maximum Weights

### Creating Random Solution

In [ ]:
def GeneateSolution(n):
    
    x = list(np.random.randint(0, 2, size=n))
    
    return x

### Cost Function

In [ ]:
def KnapsakCost(x, v, w, W):
    
    global NFE
    if NFE == None:
        NFE=0
    
    NFE += 1
    
    # Store some information
    GanedValue = np.dot(v, x)
    LostValue = np.dot(v, (1-np.array(x)))
    GanedWeight = np.dot(w, x)
    LostWeight = np.dot(w, (1-np.array(x)))
    
    violation = max(GanedWeight/W-1, 0)
    
    alpha = 10000
    
    z = LostValue + alpha*violation
    
    isFeasible = violation == 0
    
    sol = {"GanedValue": GanedValue,
           "LostValue": LostValue,
           "GanedWeight": GanedWeight,
           "LostWeight": LostWeight,
           "isFeasible": isFeasible
    }
    
    return z, sol

### Now We Use All Functions From GA Binary

In [ ]:
# Sort the population and cost (based on the cost)
def pop_sort(p, c, s):
    li = []
    for i in range(len(c)):
        li.append([c[i],i])
        
    li.sort()
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    positions, cost, sol = [], [], []
    for i in sort_index:
        positions.append(p[i])
        cost.append(c[i])
        sol.append(s[i])
        
    return positions, cost, sol

### Roullete Wheele Selection

In [ ]:
def rouletteWheelSelection(p):
    r = random.random()
    
    c = np.cumsum(p)
    
    indexes = [
    index for index in range(len(c))
    if c[index] > r
    ]
    
    return indexes[0]

### CrossOver

In [ ]:
# Uniform Crossover is better than double point crossover better than single point crossover

# Single point crossover
def singlePoint_crossover(x1, x2):
    index = int(np.random.randint(1, len(x1)-1, size=1))
    
    y1 = x1[:index] + x2[index:]
    y2 = x2[:index] + x1[index:]
    
    return y1, y2

# Double Point Crossover
def doublePoint_crossover(x1, x2):
    ind = random.sample(range(1, len(x1)-1), 2)
    
    index1 = min(ind)
    index2 = max(ind)
    
    # Another way is to generate sequence from, 1 to len(x1)-1 then shuffle it
    # Then select first two elements (it won't be the same at all) --> my_ind = list(range(1, len(x1)-1))
    # random.shuffle(my_list)
    y1 = x1[:index1] + x2[index1:index2] + x1[index2:]
    y2 = x2[:index1] + x1[index1:index2] + x2[index2:]
    
    return y1, y2

# Uniform Crossover
def uniform_crossover(x1, x2):
    alpha = list(np.random.randint(2, size=len(x1)))
    
    y1 = list(np.multiply(alpha, x1) + (1-np.array(alpha)) * np.array(x2))
    y2 = list(np.multiply(alpha, x2) + (1-np.array(alpha)) * np.array(x1))
    
    return y1, y2

def CrossOver(x1, x2):
    
    pSinglePoint = 0.1
    pDoublePoint = 0.2
    pUniform = 1-pSinglePoint-pDoublePoint
    
    METHOD = rouletteWheelSelection([pSinglePoint, pDoublePoint, pUniform])
    
    if METHOD == 0:
        y1, y2 = singlePoint_crossover(x1, x2)
    elif METHOD == 1:
        y1, y2 = doublePoint_crossover(x1, x2)
    elif METHOD == 2:
        y1, y2 = uniform_crossover(x1, x2)
    
    return y1, y2

### Mutation

In [ ]:
def Mutation(x):
    index = int(np.random.randint(0, len(x), size=1))
    
    y = x.copy()
    
    y[index] = 1-x[index]
    
    return y

In [ ]:
### Problem Parameters Definition ###
nVar = len(v)       # Number of decision variables

global NFE
NFE = 0

### GA Parameters ###
maxIt = 100     # Maximum numner of iterations
nPop = 20       # Population size 

pc = 0.8                   # Crossover percentage
nc = 2*round(pc*nPop/2)    # Number of offsprings (parents)

pm = 0.3                   # Mutation percentage
nm = round(pm*nPop)        # Number of mutants2 = unifrnd(0,2 = unifrnd(0,

### Initialization ###
pop, costs, sol = [], [], []

for i in range(0, nPop):
    pop.append(GeneateSolution(nVar))
#     costs.append(MinOne(pop[i]))
    c, s = KnapsakCost(pop[i], v, w, W)
    costs.append(c)
    sol.append(s)

# Sort the population and costs
pop, costs, sol = pop_sort(pop, costs, sol)

#  Store the best solution
bestSolution = [pop[0]]

# Array to hold best cost values
bestCosts = [costs[0]]

# Store the NFE into the array
nfe = [NFE]

### Main Loop ###
for it in range(1, maxIt):
    
    # Crossover
    popc, popc_cost = [], []
    for k in range(1, int(nc/2)):
        
        # Select parent indices
        rand1 = int(np.random.randint(nPop, size=1))
        rand2 = int(np.random.randint(nPop, size=1))
        
        # Select parents
        p1 = pop[rand1]
        p2 = pop[rand2]
        
        # Apply crossover
#         y1, y2 = singlePoint_crossover(p1, p2)
#         y1, y2 = doublePoint_crossover(p1, p2)
#         y1, y2 = uniform_crossover(p1, p2)

        y1, y2 = CrossOver(p1, p2)
        
        # Store the offspring after crossover
        popc.append(y1)
        popc.append(y2)
        
        # Evaluate the offspring
        c, s = KnapsakCost(y1, v, w, W) # v is values / w is weights / W is total weights
        costs.append(c)
        sol.append(s)
        
        c, s = KnapsakCost(y2, v, w, W)
        costs.append(c)
        sol.append(s)
#         popc_cost.append(MinOne(y1))
#         popc_cost.append(MinOne(y2))
        
    # Mutation
    popm, popm_cost = [], []
    for k in range(1, nm):
        
        # Select parent
        rand = int(np.random.randint(nPop, size=1))
        p = pop[rand]
        
        # Apply Mutation
        popm.append(Mutation(p))
        
        # Evaluate the offspring
#         popm_cost.append(MinOne(popm[-1]))
        c, s = KnapsakCost(popm[-1], v, w, W)
        costs.append(c)
        sol.append(s)
        
    # Create merged population
    pop = pop + popm + popc
    costs = costs + popm_cost + popc_cost
    
    # sort the whole population
    pop, costs, sol = pop_sort(pop, costs, sol)
    
    # Truncation
    pop = pop[:nPop]
    costs = costs[:nPop]
    sol = sol[:nPop]
    
    # Store the best solution
    bestSolution.append(pop[0])
    
    # Store the best cost
    bestCosts.append(costs[0])
    
    # Append NFE to the array
    nfe.append(NFE)
        
    print(f'Iteration {it} : NFE = {nfe[-1]}, Best Cost = {bestCosts[it]}')

### Plot the Result

In [ ]:
# Plot the result
plt.plot(nfe, bestCosts, linewidth = 3)
plt.xlabel('NFE')
plt.ylabel('Costs')

In [ ]:
# Plot the result in logarithm format
plt.plot(nfe, np.log(bestCosts), linewidth = 3)
plt.xlabel('NFE')
plt.ylabel('Costs')

In [ ]:
bestSolution[0]

In [ ]:
sol[0]